In [ ]:
# Welcome to your new notebook MVP-F1 Estatísticas Data_Lake
# Type here in the cell editor to add code!
# Camadas MVP_F1 75 Anos >> Bronze, Prata e Ouro.

In [2]:
df = spark.sql("SELECT * FROM MVP_F1.constructor_results WHERE constructorId = 6")
display(df)

StatementMeta(, cda14283-7069-4907-98f7-2e121b84008b, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cbcdb6fd-fe6b-4873-8fb8-7889791e3a5d)

In [ ]:
import dlt
from pyspark.sql.functions import col, expr, to_timestamp

delta_table_path = "Tables/constructor_results" # Caminho da sua tabela Delta Bronze

@dlt.table(
    name="f1_silver_teams_standings", # Nome da tabela Silver
    table_properties={'quality': 'silver', 'delta.columnMapping.mode': 'name'}
)
def f1_silver_teams():
    bronze_df = dlt.read_table(delta_table_path)
    # Supondo que a tabela Bronze tenha colunas como 'raceId', 'constructorId', 'points', 'year' etc.

    # Realize transformações para a camada Silver
    # Exemplo: Calcular a pontuação total por equipe em cada corrida
    silver_df = bronze_df.groupBy("year", "constructorId", "raceId") \
        .agg(expr("sum(points) as total_points_race")) \
        .withColumnRenamed("constructorId", "team_id") \
        .withColumnRenamed("raceId", "race_id") \
        .withColumnRenamed("year", "season_year")

    # Você pode adicionar mais transformações aqui, como:
    # - Filtragem de dados relevantes
    # - Junção com outras tabelas de dimensão (se existirem na sua camada Bronze ou Silver)
    # - Conversão de tipos de dados
    # - Remoção de colunas desnecessárias
    # - Padronização de nomes de colunas

    return silver_df

@dlt.table(
    name="f1_silver_drivers_standings", # Nome da tabela Silver
    table_properties={'quality': 'silver', 'delta.columnMapping.mode': 'name'}
)
def f1_silver_drivers():
    bronze_df = dlt.read_table(delta_table_path)
    # Supondo que a tabela Bronze tenha colunas como 'raceId', 'driverId', 'points', 'year' etc.

    # Realize transformações para a camada Silver
    # Exemplo: Calcular a pontuação total por piloto em cada corrida
    silver_df = bronze_df.groupBy("year", "driverId", "raceId") \
        .agg(expr("sum(points) as total_points_race")) \
        .withColumnRenamed("driverId", "driver_id") \
        .withColumnRenamed("raceId", "race_id") \
        .withColumnRenamed("year", "season_year")

    # Adicione mais transformações conforme necessário
    return silver_df

# Exemplo de como você poderia criar uma tabela Silver com informações de corrida
@dlt.table(
    name="f1_silver_races",
    table_properties={'quality': 'silver', 'delta.columnMapping.mode': 'name'}
)
def f1_silver_races_info():
    bronze_df = dlt.read_table(delta_table_path)
    # Supondo que a tabela Bronze contenha informações sobre as corridas
    # Você pode precisar selecionar colunas específicas e talvez realizar alguma limpeza
    silver_df = bronze_df.select("raceId", "year", "round", "circuitId", "name", "date", col("time").alias("race_time")) \
        .withColumnRenamed("raceId", "race_id") \
        .withColumnRenamed("year", "season_year") \
        .withColumn("race_timestamp", to_timestamp(col("date"), "yyyy-MM-dd")) # Exemplo de conversão de tipo

    return silver_df

In [3]:
df = spark.sql("SELECT raceId, constructorId, points FROM MVP_F1.constructor_results WHERE constructorId = 6")
display(df)

StatementMeta(, cda14283-7069-4907-98f7-2e121b84008b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c97ff59e-763c-465d-9d62-1f41c95b3878)

In [ ]:
df = spark.sql("SELECT * FROM MVP_F1.constructor_standings LIMIT 1000")
display(df)

In [ ]:
delta_table_path = "Tables/f1_24" # fill in your delta path
df.write.format("delta").mode("overwrite").save(delta_table_path)

import dlt
@dlt.table(table_properties={'quality': 'bronze', 'delta.columnMapping.mode': 'name', 'delta.minReaderVersion': '2', 'delta.minWriterVersion': '5'})
def netsuite_items_inventory_price():
  return (
     spark.readStream.format('cloudFiles')
     .option('cloudFiles.format', 'json')
     .option('delta.columnMapping.mode', 'name')
     .load('s3://data-lake-raw-injestion/Netsuite/Item/testLoad')
    )

In [4]:
df = spark.read.format("csv").option("header","true").load("Files/F1_24.csv")
display(df)

StatementMeta(, 6929dddf-ba75-49b5-8f6f-7033651bc7de, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f7e01887-7b0f-41b2-a487-a3df8bd877ef)

**MVP-F1 >> Qual o Ranking de Vitórias por Pilotos ?**

In [1]:
%%sql
select d.driverId, d.driverRef, d.forename, d.surname, d.nationality as Pais, count(*) as Vitorias 
from results r
join drivers d ON d.driverId = r.driverId
where r.position = 1 
Group by d.driverId, d.driverRef, d.forename, d.surname, Pais, r.position order by Vitorias desc

StatementMeta(, 01cd6e28-663a-4f55-9b4c-a3ee5e9d8a4e, 2, Finished, Available, Finished)

<Spark SQL result set with 115 rows and 6 fields>

**Quais são todas as Equipes que participaram na F1 ao longo de 75 anos?******

In [1]:
%%sql
select * from constructors

StatementMeta(, 1b5b98ca-6034-4b82-b61f-9ed8459e33c7, 2, Finished, Available, Finished)

<Spark SQL result set with 212 rows and 5 fields>

In [2]:
%%sql
select * from constructor_results where status <> '/N'

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 61, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 5 fields>

In [ ]:
df = spark.sql("SELECT * FROM MVP_F1.pit_stops LIMIT 1000")
display(df)

In [ ]:
%%sql
select * from constructor_standings where position = 1

select c.name As Equipe, ra.name AS GP, count(*) as Vitorias
from constructor_standings c
join constructors c ON c.constructorId = r.constructorId
join races ra ON ra.raceId = r.raceId
where r.position = 1
GROUP BY c.name, ra.name
order by Vitorias desc

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 2, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>

In [3]:
%%sql
select * from constructor_standings where constructorId = 6

StatementMeta(, 1b5b98ca-6034-4b82-b61f-9ed8459e33c7, 4, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>

In [ ]:
%%sql
select * from results where constructorId = 6 and position = 1

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 20, Finished, Available, Finished)

<Spark SQL result set with 249 rows and 18 fields>

**Quais são Todos os Status de Telemetria?**

In [5]:
%%sql
select * from status

StatementMeta(, 1b5b98ca-6034-4b82-b61f-9ed8459e33c7, 6, Finished, Available, Finished)

<Spark SQL result set with 139 rows and 2 fields>

**MVP-F1 >> Qual o Ranking de Vitórias por Construtores?**

In [9]:
%%sql

select c.name As Equipe, count(*) as Vitorias
from results r
join constructors c ON c.constructorId = r.constructorId
where r.position = 1 -- and constructorId = 6
GROUP BY c.name
order by Vitorias desc


StatementMeta(, d3d5b2ab-c577-4c57-bc89-f80134532451, 10, Finished, Available, Finished)

<Spark SQL result set with 47 rows and 2 fields>

**MVP-F1 >> Qual o Número de Vitórias GPs da FERRARI?**

In [10]:
%%sql

select c.name As Equipe, ra.name AS GP, count(*) as Vitorias
from results r
join constructors c ON c.constructorId = r.constructorId
join races ra ON ra.raceId = r.raceId
where r.position = 1 and r.constructorId = 6
GROUP BY c.name, ra.name
order by Vitorias desc

StatementMeta(, d3d5b2ab-c577-4c57-bc89-f80134532451, 11, Finished, Available, Finished)

<Spark SQL result set with 31 rows and 3 fields>

**MVP-F1 >> Estatistica > Qual o Número de Vitórias GPs por Pilotos ?**

In [ ]:
%%sql

select ra.name AS GP, d.driverRef As Piloto, count(*) as Vitorias
from results r
join drivers d ON d.driverId = r.driverId
join races ra ON ra.raceId = r.raceId
where r.position = 1
GROUP BY ra.name, d.driverRef
order by ra.name, Vitorias desc

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 23, Finished, Available, Finished)

<Spark SQL result set with 661 rows and 3 fields>

***MVP-F1 >> Qual o Número de Vitórias GPs por Ano ?****

In [ ]:
%%sql 

select d.driverRef As Piloto, ra.year as Ano, count(*) as Vitorias
from results r
join drivers d ON d.driverId = r.driverId
join races ra ON ra.raceId = r.raceId
where r.position = 1
GROUP BY d.driverRef,  ra.year 
order by ra.year desc

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 19, Finished, Available, Finished)

<Spark SQL result set with 403 rows and 3 fields>

In [27]:
%%sql
select * from  results where driverId = 102

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 28, Finished, Available, Finished)

<Spark SQL result set with 162 rows and 18 fields>

In [28]:
%%sql
select * from  driver_standings where driverId = 102

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 29, Finished, Available, Finished)

<Spark SQL result set with 48 rows and 7 fields>

***MVP-F1 >> Qual o Ranking Pontuação do Campeonato por Pilotos por Ano?****

In [38]:
%%sql 

select ra.year as Ano, d.driverRef As Piloto, SUM(Points) as Pontos
from results r
join drivers d ON d.driverId = r.driverId
join races ra ON ra.raceId = r.raceId
GROUP BY ra.year, d.driverRef
order by ra.year DESC , Pontos desc

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 40, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 3 fields>

***MVP-F1 >> Classificação Pilotos por Ano****

In [9]:
%%sql 

select ra.year as Ano, d.driverRef As Piloto, SUM(r.points) as Pontos
from results r
join drivers d ON d.driverId = r.driverId
join races ra ON ra.raceId = r.raceId
GROUP BY ra.year, d.driverRef
order by ra.year DESC , Pontos desc

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 10, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 3 fields>

***MVP-F1 >> Classificação Construtores por Ano****


In [91]:
%%sql 

select ra.year as Ano, c.constructorRef As Equipe, SUM(r.points) as Pontos
from results r
join constructors c ON c.constructorId = r.constructorId
join races ra ON ra.raceId = r.raceId
GROUP BY ra.year, c.constructorRef
order by ra.year DESC , Pontos desc

StatementMeta(, 5d60e120-5d4a-4dc1-af73-d19c4738d3f7, 93, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 3 fields>

In [ ]:
df = spark.sql("SELECT * FROM MVP_F1.results LIMIT 1000")
display(df)

***MVP-F1 >> Ranking de Pilotos Campeões****

In [ ]:
%%sql

SELECT
    Ano,
    Piloto,
    Pontos
FROM (
    SELECT
        ra.year AS Ano,
        d.driverRef AS Piloto,
        SUM(r.points) AS Pontos,
        ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
    FROM
        results r
    JOIN
        drivers d ON d.driverId = r.driverId
    JOIN
        races ra ON ra.raceId = r.raceId
    GROUP BY
        ra.year,
        d.driverRef
) AS ranked_results
WHERE rn = 1
ORDER BY Ano DESC;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 49, Finished, Available, Finished)

<Spark SQL result set with 75 rows and 3 fields>

In [ ]:
%%sql

SELECT
    Ano,
    Piloto,
    Pontos
FROM (
    SELECT
        ra.year AS Ano,
        d.driverRef AS Piloto,
        SUM(r.points) AS Pontos,
        ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
    FROM
        results r
    JOIN
        drivers d ON d.driverId = r.driverId
    JOIN
        races ra ON ra.raceId = r.raceId
    GROUP BY
        ra.year,
        d.driverRef
) AS ranked_results
WHERE rn = 1
ORDER BY Ano DESC;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 40, Finished, Available, Finished)

<Spark SQL result set with 75 rows and 3 fields>

**Estatistica de Pilotos mais Vitoriosos**

In [ ]:
%%sql

SELECT
    Piloto,
    COUNT(*) AS Pilotos_Campeoes
FROM (
    SELECT
        Ano,
        Piloto
    FROM (
        SELECT
            ra.year AS Ano,
            d.driverRef AS Piloto,
            SUM(r.points) AS Pontos,
            ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
        FROM
            results r
        JOIN
            drivers d ON d.driverId = r.driverId
        JOIN
            races ra ON ra.raceId = r.raceId
        GROUP BY
            ra.year,
            d.driverRef
    ) AS RankedResultsPorAno
    WHERE rn = 1
) AS PilotosMaisVitoriososPorAno
GROUP BY
    Piloto
ORDER BY
    Pilotos_Campeoes DESC;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 53, Finished, Available, Finished)

<Spark SQL result set with 33 rows and 2 fields>

In [24]:
%%sql 

select ra.year as Ano, d.driverRef As Piloto, SUM(r.points) as Pontos
from results r
join drivers d ON d.driverId = r.driverId
join races ra ON ra.raceId = r.raceId
GROUP BY ra.year, d.driverRef
ORDER by ra.year DESC , Pontos desc

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 39, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 3 fields>

**Ranking de Construtores Campeões**

In [ ]:
%%sql

SELECT
    Ano,
    Equipe,
    Pontos
FROM (
    SELECT
        ra.year AS Ano,
        c.constructorRef AS Equipe,
        SUM(r.points) AS Pontos,
        ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
    FROM
        results r
    JOIN
        constructors c ON c.constructorId = r.constructorId
    JOIN
        races ra ON ra.raceId = r.raceId
    GROUP BY
        ra.year,
        c.constructorRef
) AS RankedResults
WHERE rn = 1
ORDER BY Ano DESC;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 42, Finished, Available, Finished)

<Spark SQL result set with 75 rows and 3 fields>

**Estatistica de Construtores mais Vitoriosos**

In [ ]:
%%sql

SELECT
    Equipe,
    COUNT(*) AS Trofeu_Construtores
FROM (
    SELECT
        Ano,
        Equipe
    FROM (
        SELECT
            ra.year AS Ano,
            c.constructorRef AS Equipe,
            SUM(r.points) AS Pontos,
            ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
        FROM
            results r
        JOIN
            constructors c ON c.constructorId = r.constructorId
        JOIN
            races ra ON ra.raceId = r.raceId
        GROUP BY
            ra.year,
            c.constructorRef
    ) AS RankedResultsPorAno
    WHERE rn = 1
) AS EquipesMaisVitoriosasPorAno
GROUP BY
    Equipe
ORDER BY
    Trofeu_Construtores DESC;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 46, Finished, Available, Finished)

<Spark SQL result set with 18 rows and 2 fields>

**Estatística de Campeonato de Construtores por Ano**

In [ ]:
%%sql

SELECT
    Equipe,
    Ano_Trofeu
FROM (
    SELECT
        ra.year AS Ano_Trofeu,
        c.constructorRef AS Equipe,
        SUM(r.points) AS Pontos,
        ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
    FROM
        results r
    JOIN
        constructors c ON c.constructorId = r.constructorId
    JOIN
        races ra ON ra.raceId = r.raceId
    GROUP BY
        ra.year,
        c.constructorRef
) AS RankedResultsPorAno
WHERE rn = 1
ORDER BY Equipe, Ano_Trofeu;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 60, Finished, Available, Finished)

<Spark SQL result set with 75 rows and 2 fields>

In [ ]:
%%sql

SELECT
    Ano,
    Piloto,
    Pontos
FROM (
    SELECT
        ra.year AS Ano,
        d.driverRef AS Piloto,
        SUM(r.points) AS Pontos,
        ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY SUM(r.points) DESC) AS rn
    FROM
        results r
    JOIN
        drivers d ON d.driverId = r.driverId
    JOIN
        races ra ON ra.raceId = r.raceId
    GROUP BY
        ra.year,
        d.driverRef
) AS ranked_results
WHERE rn = 1
ORDER BY Ano DESC;

StatementMeta(, 70a8467f-fceb-4232-87ac-abdb987bf28d, 54, Finished, Available, Finished)

<Spark SQL result set with 75 rows and 3 fields>

******Estatistica de Vitorias de Senna durante os 10 anos que Pilotou de 1984 a 1994**

In [24]:
%%sql 

SELECT d.driverRef As Piloto, c.name as Equipe, ra.name as Corrida, cr.name as Circuito, ra.year as Ano, count(*) as Vitorias
FROM results r
JOIN drivers d ON d.driverId = r.driverId
JOIN races ra ON ra.raceId = r.raceId
JOIN circuits cr ON cr.circuitId = ra.circuitId
JOIN constructors c ON c.constructorId = r.constructorId
WHERE r.position = 1 and d.driverId = 102
GROUP BY d.driverRef, c.name, ra.name, cr.name, ra.year 
ORDER by ra.year, c.name desc

StatementMeta(, 95fe1958-3092-435a-ac0c-a2e376f949d9, 25, Finished, Available, Finished)

<Spark SQL result set with 41 rows and 6 fields>